In [ ]:
import pandas as pd
import sqlite3
import altair as alt
import geopandas as gpd
from vega_datasets import data

In [ ]:
db_name = 'field_crops.db'
crop_table = 'midwest_key_field_crops_cleaned'
area_table = 'midwest_area_planted_cleaned'
conn = sqlite3.connect(db_name) 

In [ ]:
color_scale = alt.Scale(domain=['CORN', 'SOYBEANS', 'WHEAT'],
                        range=['#FFB14E', '#FA8775', '#B5E384'])

In [ ]:
output_path = 'static_final/'

# Regional Analysis

## Total Production

In [ ]:
query = f"""
Select 
    commodity_desc,
    year, 
    sum(value) as total_prod
from {crop_table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year >= 1975
group by 
    commodity_desc, year
"""
agg_prod_region_all_years = pd.read_sql(query, conn)

chart = alt.Chart(agg_prod_region_all_years).mark_area().encode(
    x=alt.X('year:O', axis=alt.Axis(title='Year')),
    y=alt.Y('total_prod:Q', axis=alt.Axis(title='Total Production (in BUs)')),
    color=alt.Color('commodity_desc:N', scale=color_scale, legend=alt.Legend(title="Commodity"))
).properties(
    title=alt.TitleParams(
        text='Total Crop Production Over Time',
        subtitle='Corn, Soybeans, and Wheat (in BU)',
        anchor='middle'
    ),
    width=600,
    height=400
)

file_name = '01_AGG_PRODUCTION_BY_CROP'
chart.save(f'{output_path}{file_name}.png')


## Total Area Planted


In [ ]:
query = f"""
Select 
    commodity_desc,
    year, 
    sum(value) as total_area_planted,
    count(*) as num_counties
from {area_table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year >= 1975
group by 
    commodity_desc, year
"""
agg_area_planted_region_all_years = pd.read_sql(query, conn)

chart = alt.Chart(agg_area_planted_region_all_years).mark_area().encode(
    x=alt.X('year:O', axis=alt.Axis(title='Year')),
    y=alt.Y('total_area_planted:Q', axis=alt.Axis(title='Total Area Planted (in Acres)')),
    color=alt.Color('commodity_desc:N', scale=color_scale, legend=alt.Legend(title="Commodity"))   
    ).properties(
        title=alt.TitleParams(
            text='Total Area Planted Over Time in Acres',
            subtitle='Corn, Soybeans, and Wheat',
            anchor='middle'
        ),
        width=600,
        height=400
    )

chart

## aggregate productivity.

In [ ]:
agg_yield_region_all_years = pd.merge(agg_area_planted_region_all_years, agg_prod_region_all_years, on=['commodity_desc', 'year'])
agg_yield_region_all_years['yield'] = agg_yield_region_all_years['total_prod'] / agg_yield_region_all_years['total_area_planted']

agg_yield_region_all_years

chart = alt.Chart(agg_yield_region_all_years).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(title='Year')),
    y=alt.Y('yield:Q', axis=alt.Axis(title='Yield')),
    color=alt.Color('commodity_desc:N', scale=color_scale, legend=alt.Legend(title="Crop"))   
    ).properties(
        title='Aggregate Yield Over Time',
        width=600,
        height=400
    )

chart

# State Analysis


## Production

### Aggregated by State

### Production Change Bar

In [ ]:
query = f"""
Select 
    avg(value) AS pres_prod,
    state_alpha
from {crop_table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 2018 and 2023
group by state_alpha
"""
conn = sqlite3.connect(db_name) 
avg_prod_2015_2020 = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS past_prod,
    state_alpha
from {crop_table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 1975 and 1980
group by state_alpha
"""
avg_prod_1975_1980 = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS pres_area,
    state_alpha
from {area_table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 2018 and 2023
group by state_alpha
"""
avg_area_2015_2020 = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS past_area,
    state_alpha
from {area_table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 1975 and 1980
group by state_alpha
"""
avg_area_1975_1980 = pd.read_sql(query, conn)

avg_yield_past = pd.merge(avg_area_1975_1980, avg_prod_1975_1980, on=["state_alpha"])
avg_yield_past["yield_past"] = (avg_yield_past['past_prod'] / avg_yield_past['past_area']) 

avg_yield_present = pd.merge(avg_area_2015_2020, avg_prod_2015_2020, on=["state_alpha"])
avg_yield_present["yield_present"] = (avg_yield_present['pres_prod'] / avg_yield_present['pres_area']) 

yield_change = pd.merge(avg_yield_past, avg_yield_present, on=[ "state_alpha"])

yield_change["abs_change_yield"] = (yield_change['yield_present'] -  yield_change['yield_past']) 
yield_change["perc_change_yield"] = ((yield_change['yield_present'] -  yield_change['yield_past']) / yield_change['yield_past'])*100
yield_change


query = f"""
Select 
    avg(value) AS Value_20,
    commodity_desc,
    state_alpha
from {crop_table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 2018 and 2023
group by state_alpha, commodity_desc
"""
conn = sqlite3.connect(db_name) 
avg_prod_2015_2020 = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS Value_70,
    commodity_desc,
    state_alpha
from {crop_table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 1975 and 1980
group by state_alpha, commodity_desc
"""
avg_prod_1975_1980 = pd.read_sql(query, conn)

prod_change = pd.merge(avg_prod_2015_2020, avg_prod_1975_1980, on=["commodity_desc", "state_alpha"])
prod_change["abs_change_in_prod"] = (prod_change['Value_20'] -  prod_change['Value_70']) 

# Calculate the total production change per state and sort by this total
state_totals = prod_change.groupby('state_alpha')['abs_change_in_prod'].sum().sort_values(ascending=False)
prod_change['state_alpha'] = pd.Categorical(prod_change['state_alpha'], categories=state_totals.index, ordered=True)

# Create the bar chart with sorted states (highest total on the left)
bar_chart = alt.Chart(prod_change).mark_bar().encode(
    x=alt.X('state_alpha:O', title='State', sort=state_totals.index.tolist()),
    y=alt.Y('abs_change_in_prod:Q', title='Change in Production (in BU)'),
    color=alt.Color('commodity_desc:N', scale=color_scale, legend=alt.Legend(title="Commodity")), 
    xOffset='commodity_desc:N'
)
line_chart = alt.Chart(yield_change).mark_line(color='red').encode(
    x=alt.X('state_alpha:O', title='State', sort=state_totals.index.tolist()),
    y=alt.Y('abs_change_yield:Q', title='Change in Yield Over Period'),
)
combined_chart = alt.layer(bar_chart, line_chart).resolve_scale(
    y='independent'
).properties(
    title=alt.TitleParams(
        text='Absolute Change in Production Over Time in BU',
        anchor='middle'
    ),
    width=600,
    height=400
)
combined_chart


# County Level


In [ ]:
states = data.us_10m.url  # URL for U.S. states
counties = data.us_10m.url  # URL for U.S. counties
states_gdf = gpd.read_file(states)
counties_gdf = gpd.read_file(counties)

## Yield Maps

In [ ]:
query = f"""
Select 
    avg(value) AS avg_prod_present,
    commodity_desc,
    state_alpha, 
    state_ansi|| county_ansi as id
from {crop_table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 2018 and 2023
group by state_ansi|| county_ansi, commodity_desc
"""
avg_prod_present = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS avg_prod_past,
    commodity_desc,
    state_alpha,
    state_ansi|| county_ansi as id
from {crop_table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 1975 and 1980
group by state_ansi|| county_ansi, commodity_desc
"""
avg_prod_past = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS avg_area_present,
    commodity_desc,
    state_alpha,
    state_ansi|| county_ansi as id
from {area_table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 2018 and 2023
group by state_ansi|| county_ansi , commodity_desc
"""
avg_area_present = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS avg_area_past,
    commodity_desc,
    state_alpha,
    state_ansi|| county_ansi as id
from {area_table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 1975 and 1980
group by state_ansi|| county_ansi , commodity_desc
"""
avg_area_past = pd.read_sql(query, conn)

avg_yield_past = pd.merge(avg_prod_past, avg_area_past, on=["commodity_desc", "id", "state_alpha"])
avg_yield_past["yield_past"] = (avg_yield_past['avg_prod_past'] / avg_yield_past['avg_area_past']) 

avg_yield_present = pd.merge(avg_prod_present, avg_area_present, on=["commodity_desc", "id", "state_alpha"])
avg_yield_present["yield_present"] = (avg_yield_present['avg_prod_present'] / avg_yield_present['avg_area_present']) 

yield_change = pd.merge(avg_yield_past, avg_yield_present, on=["commodity_desc", "id", "state_alpha"])

yield_change["abs_change_yield"] = (yield_change['yield_present'] -  yield_change['yield_past']) 
yield_change["perc_change_yield"] = ((yield_change['yield_present'] -  yield_change['yield_past']) / yield_change['yield_past'])*100

In [ ]:
query = f"""
Select 
    distinct
    state_ansi
from {crop_table} 
"""
conn = sqlite3.connect(db_name) 
check = pd.read_sql(query, conn)

state_ansi_list = check.iloc[:,0].to_list()
midwest_counties_gdf = counties_gdf[counties_gdf['id'].str[:2].isin(state_ansi_list)]
midwest_counties_gdf = midwest_counties_gdf[
    counties_gdf['id'].str[:2].isin(state_ansi_list) & 
    (counties_gdf['id'].str.len() == 5)  
]


merged = gpd.GeoDataFrame(pd.merge(yield_change, midwest_counties_gdf, on='id', how='left'))
merged.set_geometry('geometry', inplace=True)


crop_list = [ 'CORN', 'SOYBEANS', 'WHEAT']

# Load U.S. states data
states = alt.topo_feature(data.us_10m.url, 'states')

# Define state IDs for the Midwestern states
midwestern_state_ids = [17, 18, 19, 20, 26, 27, 29, 31, 38, 39, 46, 55]

# Filter the background chart for the selected states and add black borders
state_map_background = alt.Chart(states).mark_geoshape(
    fill=None,
    stroke='black',  # Set border color to black
    strokeWidth=1.5  # Adjust width as needed
).transform_filter(
    alt.FieldOneOfPredicate(field='id', oneOf=midwestern_state_ids)
).properties(
    width=800,
    height=500
).project('albersUsa')

for crop in crop_list:
    crop_df = merged[merged['commodity_desc']== crop]

    # Define the background chart with a gray fill and black stroke for county borders
    county_map_background = alt.Chart(midwest_counties_gdf).mark_geoshape(
        fill='lightgray',  # Background color
        stroke='black',    # Outline color for counties
        strokeWidth=0.5    # Thickness of county borders
    ).properties(
        width=800,
        height=500
    ).project('albersUsa')  # Use Albers USA projection

    # Define the filled map chart
    county_map_filled = alt.Chart(crop_df).mark_geoshape(
        stroke='black',   # Outline color for counties
        strokeWidth=0.5   # Thickness of county borders
    ).encode(
        color=alt.Color('abs_change_yield:Q',       
                        scale=alt.Scale(
                        scheme='redblue',        # Sequential color scale for yield change       # Adjust domain to match data range for more contrast
        )),  # Sequential color scale for the 'value' column
        tooltip=['id:N', 'abs_change_yield:Q']  # Tooltip with county ID and value
    ).properties(
        title=f'Map of Yield Change for {crop}',
        width=800,
        height=500
    ).project('albersUsa')  # Use Albers USA projection

    # Layer the filled map on top of the gray background
    layered_map = county_map_background + county_map_filled + state_map_background

    # Display the chart
    layered_map.show()

## Climate Data

In [ ]:
# Filters
noaa_midwest_codes = ["11", "12", "13", "14", "20", "21", "23", "25", "32", "33", "39", "47"]

fips_mapping = {
    "11": "17",  # Illinois
    "12": "18",  # Indiana
    "13": "19",  # Iowa
    "14": "20",  # Kansas
    "20": "26",  # Michigan
    "21": "27",  # Minnesota
    "23": "29",  # Missouri
    "25": "31",  # Nebraska
    "32": "38",  # North Dakota
    "33": "39",  # Ohio
    "39": "46",  # South Dakota
    "47": "55"   # Wisconsin
}

final_df_cols = ['Year', 'County_Code', 'state_fips']

def parse_climdiv_data(file_path, yearly_avg_column_name, midwest_codes=noaa_midwest_codes, final_df_cols=final_df_cols):
    # Define the column widths based on the provided positions
    column_specs = [
        (0, 2),    # STATE-CODE (1-2)
        (2, 5),    # DIVISION-NUMBER (3-5)
        (5, 7),    # ELEMENT CODE (6-7)
        (7, 11),   # YEAR (8-11)
        (11, 18),  # JAN-VALUE (12-18)
        (18, 25),  # FEB-VALUE (19-25)
        (25, 32),  # MAR-VALUE (26-32)
        (32, 39),  # APR-VALUE (33-39)
        (39, 46),  # MAY-VALUE (40-46)
        (46, 53),  # JUNE-VALUE (47-53)
        (53, 60),  # JULY-VALUE (54-60)
        (60, 67),  # AUG-VALUE (61-67)
        (67, 74),  # SEPT-VALUE (68-74)
        (74, 81),  # OCT-VALUE (75-81)
        (81, 88),  # NOV-VALUE (82-88)
        (88, 95),  # DEC-VALUE (89-95)
    ]

    # Column names
    column_names = [
        "State_Code", "Division_Number", "Element_Code", "Year",
        "Jan_Value", "Feb_Value", "Mar_Value", "Apr_Value", "May_Value", 
        "Jun_Value", "Jul_Value", "Aug_Value", "Sep_Value", "Oct_Value", 
        "Nov_Value", "Dec_Value"
    ]
    
    # Read the fixed-width file, treating State_Code and Division_Number as strings
    df = pd.read_fwf(file_path, colspecs=column_specs, names=column_names, 
                     dtype={"State_Code": str, "Division_Number": str})

    
    # Create a new 'state_fips' column that maps the State_Code using the fips_mapping dictionary
    df['state_fips'] = df['State_Code'].map(fips_mapping)  # All values are strings

    # Create a new column that combines State_Code and Division_Number
    df['County_Code'] = df['state_fips'] + df['Division_Number']

    # Convert monthly values to numeric, replacing missing indicators
    numeric_columns = column_names[4:]
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    # Handle missing values based on given missing indicators
    df.replace({
        "Jan_Value": {-99.99: None, -9.99: None},
        "Feb_Value": {-99.99: None, -9.99: None},
        "Mar_Value": {-99.99: None, -9.99: None},
        "Apr_Value": {-99.99: None, -9.99: None},
        "May_Value": {-99.99: None, -9.99: None},
        "Jun_Value": {-99.99: None, -9.99: None},
        "Jul_Value": {-99.99: None, -9.99: None},
        "Aug_Value": {-99.99: None, -9.99: None},
        "Sep_Value": {-99.99: None, -9.99: None},
        "Oct_Value": {-99.99: None, -9.99: None},
        "Nov_Value": {-99.99: None, -9.99: None},
        "Dec_Value": {-99.99: None, -9.99: None}
    }, inplace=True)

    # Calculate the yearly average, ignoring missing values
    df[yearly_avg_column_name] = df[numeric_columns].mean(axis=1)

    midwest_df = df[df['State_Code'].isin(midwest_codes)]

    midwest_df_post1960 = midwest_df[midwest_df['Year'] > 1950]

    output_columns = final_df_cols + [yearly_avg_column_name]
    
    return midwest_df_post1960[output_columns]

precipitation_path = 'data/climate_data/climdiv-pcpncy-v1.0.0-20241021.txt'
avg_temp_path = 'data/climate_data/climdiv-tmpccy-v1.0.0-20241021.txt'
max_temp_path = 'data/climate_data/climdiv-tmaxcy-v1.0.0-20241021.txt'
min_temp_path = 'data/climate_data/climdiv-tmincy-v1.0.0-20241021.txt'
precip_df = parse_climdiv_data(precipitation_path, "ann_avg_precip")
avg_temp_df = parse_climdiv_data(avg_temp_path, "ann_avg_temp")
max_temp_df = parse_climdiv_data(max_temp_path, "ann_max_temp")
min_temp_df = parse_climdiv_data(min_temp_path, "ann_min_temp")
# Merge the DataFrames one by one
merge_cols = ['Year', 'County_Code', 'state_fips']
annual_climate_data_df = precip_df.merge(avg_temp_df, on=merge_cols).merge(max_temp_df, on=merge_cols).merge(min_temp_df, on=merge_cols)

annual_climate_data_df = annual_climate_data_df.sort_values(by=['County_Code', 'Year'])

# Apply rolling mean within each group
rolling_avg_30yr_climate_data_df = (
    annual_climate_data_df
    .groupby('County_Code')[['Year', 'ann_avg_precip', 'ann_avg_temp', 'ann_max_temp', 'ann_min_temp']]
    .apply(lambda x: x.set_index('Year').rolling(window=30).mean())
    .reset_index()
)


In [ ]:
def load_midwest_counties(db_name, table, counties_gdf):
    # SQL query to get distinct state ANSI codes
    query = f"""
    SELECT 
        DISTINCT state_ansi
    FROM {table} 
    """
    
    # Connect to the database and execute the query
    with sqlite3.connect(db_name) as conn:
        check = pd.read_sql(query, conn)

    # Extract the list of state ANSI codes
    state_ansi_list = check.iloc[:, 0].to_list()
    
    # Filter the counties GeoDataFrame to include only Midwest counties
    midwest_counties_gdf = counties_gdf[
        counties_gdf['id'].str[:2].isin(state_ansi_list) & 
        (counties_gdf['id'].str.len() == 5)
    ]

    return midwest_counties_gdf

def create_climate_maps(merged_1980, midwest_counties_gdf, climate_metrics):
    
    # backgrounds
    # Define the background chart with a gray fill and black stroke for county borders
    county_map_background = alt.Chart(midwest_counties_gdf).mark_geoshape(
        fill='lightgray',  # Background color
        stroke='black',    # Outline color for counties
        strokeWidth=0.5   # Thickness of county borders
    ).properties(
        width=800,
        height=500
    ).project('albersUsa')  # Use Albers USA projection

    # Load U.S. states data
    states = alt.topo_feature(data.us_10m.url, 'states')

    # Define state IDs for the Midwestern states
    midwestern_state_ids = [17, 18, 19, 20, 26, 27, 29, 31, 38, 39, 46, 55]

    # Filter the background chart for the selected states and add black borders
    state_map_background = alt.Chart(states).mark_geoshape(
        fill=None,
        stroke='black',  # Set border color to black
        strokeWidth=1.5  # Adjust width as needed
    ).transform_filter(
        alt.FieldOneOfPredicate(field='id', oneOf=midwestern_state_ids)
    ).properties(
        width=800,
        height=500
    ).project('albersUsa')
    
    
    
    for metric in climate_metrics:
        columns = ['id', 'geometry']
        metric_df = merged_1980[columns + [metric]]

        # Define the filled map chart
        county_map_filled = alt.Chart(metric_df).mark_geoshape(
            stroke='black',   # Outline color for counties
            strokeWidth=0.5   # Thickness of county borders
        ).encode(color=alt.Color(f'{metric}:Q',       
                    scale=alt.Scale(
                    scheme='redblue',        # Sequential color scale for yield change       # Adjust domain to match data range for more contrast
        )),  # Sequential color scale for the metric
            tooltip=['id:N', f'{metric}:Q'] # Tooltip with county ID and metric value
        ).properties(
            title=f'Change in Average Annual {metric}, between 1980 and 2023',
            width=800,
            height=500
        ).project('albersUsa')  # Use Albers USA projection

        # Layer the filled map on top of the gray background
        layered_map = county_map_background + county_map_filled + state_map_background
        # Display the chart
        layered_map.show()

def calc_difference(metric, type):
    # Assuming rolling_avg_30yr_climate_data_df is your DataFrame
    # Step 1: Filter the DataFrame
    filtered_df = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([1980, 2023])]

    # Step 2: Pivot the DataFrame to make 'Year' columns
    pivot_df = filtered_df.pivot(index='County_Code', columns='Year', values=metric)

    # Step 3: Calculate the difference between 2023 and 1980
    if type == "abs_change":
        pivot_df[f'{metric}_{type}'] = pivot_df[2023] - pivot_df[1980]
    if type == "pct_change":
        pivot_df[f'{metric}_{type}'] = ((pivot_df[2023] - pivot_df[1980]) / pivot_df[1980])*100

    # Reset index if needed
    pivot_df.reset_index(inplace=True)

    # Now pivot_df contains the difference for each County_Code
    return pivot_df, f'{metric}_{type}'

def gen_change_df(climate_metrics, type):
    change_df_list = []
    for metric in climate_metrics:
        change_df, col_name = calc_difference(metric, type)
        # Append the relevant columns to the list
        change_df_list.append(change_df[['County_Code', col_name]])
    
    # Concatenate all DataFrames in the list into a single DataFrame
    merge_cols = [ 'County_Code']
    change_climate_data_df = change_df_list[0].merge(change_df_list[1], on=merge_cols).merge(change_df_list[2], on=merge_cols).merge(change_df_list[3], on=merge_cols)
    change_climate_data_df.reset_index(drop=True, inplace=True)
    return change_climate_data_df


### Change in Climate Data

In [ ]:
midwest_counties_gdf = load_midwest_counties(db_name, area_table, counties_gdf)

# Example usage
climate_metrics = ['ann_avg_precip', 'ann_avg_temp', 'ann_max_temp', 'ann_min_temp']
abs_change_climate_data_df = gen_change_df(climate_metrics,  "abs_change")
pct_change_climate_data_df = gen_change_df(climate_metrics,  "pct_change")

abs_change_climate_data_df.rename(columns={'County_Code': 'id'}, inplace=True)

# Merge the result DataFrame with the GeoDataFrame
change_climate_data_gdf = gpd.GeoDataFrame(pd.merge(abs_change_climate_data_df, midwest_counties_gdf, on='id', how='left'))

# Set the geometry for the GeoDataFrame
change_climate_data_gdf.set_geometry('geometry', inplace=True)
print(len(change_climate_data_gdf))

climate_metrics = ['ann_avg_precip_abs_change', 'ann_avg_temp_abs_change', 'ann_max_temp_abs_change', 'ann_min_temp_abs_change']
create_climate_maps(change_climate_data_gdf, midwest_counties_gdf, climate_metrics)

### Current Climate Data

In [ ]:
climate_data_2023 = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([2023])]
climate_data_2023.rename(columns={'County_Code': 'id'}, inplace=True)

change_climate_data_gdf = gpd.GeoDataFrame(pd.merge(climate_data_2023, midwest_counties_gdf, on='id', how='left'))

# Set the geometry for the GeoDataFrame
change_climate_data_gdf.set_geometry('geometry', inplace=True)
print(len(change_climate_data_gdf))

climate_metrics = ['ann_avg_precip', 'ann_avg_temp', 'ann_max_temp', 'ann_min_temp']
create_climate_maps(change_climate_data_gdf, midwest_counties_gdf, climate_metrics)


### 1980 Climate Data

In [ ]:
climate_data_1980 = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([1980])]
climate_data_1980.rename(columns={'County_Code': 'id'}, inplace=True)

change_climate_data_gdf = gpd.GeoDataFrame(pd.merge(climate_data_1980, midwest_counties_gdf, on='id', how='left'))

# Set the geometry for the GeoDataFrame
change_climate_data_gdf.set_geometry('geometry', inplace=True)
print(len(change_climate_data_gdf))

climate_metrics = ['ann_avg_precip', 'ann_avg_temp', 'ann_max_temp', 'ann_min_temp']
create_climate_maps(change_climate_data_gdf, midwest_counties_gdf, climate_metrics)


### Change Relative to optimal temp and precip

In [ ]:
climate_data_1980 = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([1980])]
climate_data_1980.rename(columns={'County_Code': 'id'}, inplace=True)

climate_data_1980['1980_temp_deviation'] = climate_data_1980['ann_avg_temp'] - 50
climate_data_1980['1980_precip_deviation'] = climate_data_1980['ann_avg_precip'] - 3
climate_data_1980 = climate_data_1980.drop(['ann_max_temp', 'ann_min_temp'], axis=1)

climate_data_2023 = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([2023])]
climate_data_2023.rename(columns={'County_Code': 'id'}, inplace=True)


climate_data_2023['2023_temp_deviation'] = climate_data_2023['ann_avg_temp'] - 50
climate_data_2023['2023_precip_deviation'] = climate_data_2023['ann_avg_precip'] - 3
climate_data_2023 = climate_data_2023.drop(['ann_max_temp', 'ann_min_temp'], axis=1)


climate_data_1980_2023 = pd.merge(climate_data_1980, climate_data_2023, on='id')
climate_data_1980_2023['temp_change_anchored'] = climate_data_1980_2023['2023_temp_deviation'] - climate_data_1980_2023['1980_temp_deviation']

climate_data_1980_2023

In [ ]:
climate_data_1980 = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([1980])]
climate_data_1980.rename(columns={'County_Code': 'id'}, inplace=True)

climate_data_1980['1980_temp_deviation'] = abs(climate_data_1980['ann_avg_temp'] - 50)
climate_data_1980['1980_precip_deviation'] = abs(climate_data_1980['ann_avg_precip'] - 3)
climate_data_1980 = climate_data_1980.drop(['ann_max_temp', 'ann_min_temp'], axis=1)

climate_data_2023 = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([2023])]
climate_data_2023.rename(columns={'County_Code': 'id'}, inplace=True)


climate_data_2023['2023_temp_deviation'] = abs(climate_data_2023['ann_avg_temp'] - 50)
climate_data_2023['2023_precip_deviation'] = abs(climate_data_2023['ann_avg_precip'] - 3)
climate_data_2023 = climate_data_2023.drop(['ann_max_temp', 'ann_min_temp'], axis=1)


climate_data_1980_2023 = pd.merge(climate_data_1980, climate_data_2023, on='id')
climate_data_1980_2023['temp_change_anchored'] = climate_data_1980_2023['2023_temp_deviation'] - climate_data_1980_2023['1980_temp_deviation']

climate_data_1980_2023

change_climate_data_gdf = gpd.GeoDataFrame(pd.merge(climate_data_1980_2023, midwest_counties_gdf, on='id', how='left'))

# Set the geometry for the GeoDataFrame
change_climate_data_gdf.set_geometry('geometry', inplace=True)
print(len(change_climate_data_gdf))

climate_metrics = ['temp_change_anchored']
create_climate_maps(change_climate_data_gdf, midwest_counties_gdf, climate_metrics)

In [ ]:
# Filter the 1980 data
climate_data_1980 = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([1980])]
climate_data_1980.rename(columns={'County_Code': 'id'}, inplace=True)

# Calculate deviation from a target temperature and precipitation
climate_data_1980['1980_temp_deviation'] = abs(climate_data_1980['ann_avg_temp'] - 50)
climate_data_1980['1980_precip_deviation'] = abs(climate_data_1980['ann_avg_precip'] - 3)

# Define a function to categorize temperatures by range distance
def temp_distance_category(temp):
    if 48 <= temp <= 52:
        return "Optimal Range (48 - 52)"
    elif 46 <= temp < 48:
        return "Between 0-2 degrees below range"
    elif 44 <= temp < 46:
        return "Between 2-4 degrees below range"
    elif 42 <= temp < 44:
        return "Between 4-6 degrees below range"
    elif 40 <= temp < 42:
        return "Between 6-8 degrees below range"
    elif temp < 40:
        return "More than 8 degrees below range"
    elif 52 < temp <= 54:
        return "Between 0-2 degrees above range"
    elif 54 < temp <= 56:
        return "Between 2-4 degrees above range"
    elif 56 < temp <= 58:
        return "Between 4-6 degrees above range"
    elif 58 < temp <= 60:
        return "Between 6-8 degrees above range"
    else:
        return "More than 8 degrees above range"

# Apply the function to create the new categorical variable
climate_data_1980['temp_distance_category'] = climate_data_1980['ann_avg_temp'].apply(temp_distance_category)

# Drop unnecessary columns
climate_data_1980 = climate_data_1980.drop(['ann_max_temp', 'ann_min_temp'], axis=1)

climate_data_1980

In [ ]:
color_scheme = {
    "Between 0-2 degrees below range": "#ADD8E6",  # Light Blue
    "Between 2-4 degrees below range": "#87CEEB",  # Sky Blue
    "Between 4-6 degrees below range": "#4682B4",  # Steel Blue
    "Between 6-8 degrees below range": "#5F9EA0",  # Cadet Blue
    "More than 8 degrees below range": "#1E90FF",  # Dodger Blue
    "Optimal Range (48 - 52)": "#3CB371",  # Lime Green,
    "Between 0-2 degrees above range": "#FFD700",  # Gold
    "Between 2-4 degrees above range": "#FFA500",  # Orange
    "Between 4-6 degrees above range": "#FF8C00",  # Dark Orange
    "Between 6-8 degrees above range": "#FF4500",  # Orange Red
    "More than 8 degrees above range": "#FF0000"   # Red
}




change_climate_data_gdf = gpd.GeoDataFrame(pd.merge(climate_data_1980, midwest_counties_gdf, on='id', how='left'))

# Set the geometry for the GeoDataFrame
change_climate_data_gdf.set_geometry('geometry', inplace=True)

county_map_background = alt.Chart(midwest_counties_gdf).mark_geoshape(
    fill='lightgray',  # Background color
    stroke='black',    # Outline color for counties
    strokeWidth=0.5   # Thickness of county borders
).properties(
    width=800,
    height=500
).project('albersUsa')  # Use Albers USA projection

# Load U.S. states data
states = alt.topo_feature(data.us_10m.url, 'states')

# Define state IDs for the Midwestern states
midwestern_state_ids = [17, 18, 19, 20, 26, 27, 29, 31, 38, 39, 46, 55]

# Filter the background chart for the selected states and add black borders
state_map_background = alt.Chart(states).mark_geoshape(
    fill=None,
    stroke='black',  # Set border color to black
    strokeWidth=1.5  # Adjust width as needed
).transform_filter(
    alt.FieldOneOfPredicate(field='id', oneOf=midwestern_state_ids)
).properties(
    width=800,
    height=500
).project('albersUsa')

# Define the filled map chart using the categorical variable
county_map_filled = alt.Chart(change_climate_data_gdf).mark_geoshape(
        stroke='black',   # Outline color for counties
        strokeWidth=0.5   # Thickness of county borders
    ).encode(color=alt.Color('temp_distance_category:N',
                    scale=alt.Scale(
                        domain=list(color_scheme.keys()),
                        range=list(color_scheme.values())
            )
        )
    ).properties(
        title='Temperature Distance from 48-52 Range in 1980',
        width=800,
        height=500
    ).project('albersUsa')  # Use Albers USA projection

# Layer the filled map on top of the gray background
layered_map =  county_map_background + county_map_filled + state_map_background

# Display the chart
layered_map.show()


In [ ]:
# Filter the 1980 data
climate_data_2023 = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([2023])]
climate_data_2023.rename(columns={'County_Code': 'id'}, inplace=True)

# Calculate deviation from a target temperature and precipitation
climate_data_2023['1980_temp_deviation'] = abs(climate_data_2023['ann_avg_temp'] - 50)
climate_data_2023['1980_precip_deviation'] = abs(climate_data_2023['ann_avg_precip'] - 3)

# Define a function to categorize temperatures by range distance
# Apply the function to create the new categorical variable
climate_data_2023['temp_distance_category'] = climate_data_2023['ann_avg_temp'].apply(temp_distance_category)

# Drop unnecessary columns
climate_data_2023 = climate_data_2023.drop(['ann_max_temp', 'ann_min_temp'], axis=1)

change_climate_data_gdf = gpd.GeoDataFrame(pd.merge(climate_data_2023, midwest_counties_gdf, on='id', how='left'))

# Set the geometry for the GeoDataFrame
change_climate_data_gdf.set_geometry('geometry', inplace=True)

county_map_background = alt.Chart(midwest_counties_gdf).mark_geoshape(
    fill='lightgray',  # Background color
    stroke='black',    # Outline color for counties
    strokeWidth=0.5   # Thickness of county borders
).properties(
    width=800,
    height=500
).project('albersUsa')  # Use Albers USA projection

# Load U.S. states data
states = alt.topo_feature(data.us_10m.url, 'states')

# Define state IDs for the Midwestern states
midwestern_state_ids = [17, 18, 19, 20, 26, 27, 29, 31, 38, 39, 46, 55]

# Filter the background chart for the selected states and add black borders
state_map_background = alt.Chart(states).mark_geoshape(
    fill=None,
    stroke='black',  # Set border color to black
    strokeWidth=1.5  # Adjust width as needed
).transform_filter(
    alt.FieldOneOfPredicate(field='id', oneOf=midwestern_state_ids)
).properties(
    width=800,
    height=500
).project('albersUsa')

# Define the filled map chart using the categorical variable
county_map_filled = alt.Chart(change_climate_data_gdf).mark_geoshape(
        stroke='black',   # Outline color for counties
        strokeWidth=0.5   # Thickness of county borders
    ).encode(color=alt.Color('temp_distance_category:N',
                    scale=alt.Scale(
                        domain=list(color_scheme.keys()),
                        range=list(color_scheme.values())
            )
        )
    ).properties(
        title='Temperature Distance from 48-52 Range in 2023',
        width=800,
        height=500
    ).project('albersUsa')  # Use Albers USA projection

# Layer the filled map on top of the gray background
layered_map =  county_map_background + county_map_filled + state_map_background

# Display the chart
layered_map.show()


## Yield x Climate Data Heat Maps

In [ ]:
climate_data_1980 = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([1980])]
climate_data_1980.rename(columns={'County_Code': 'id'}, inplace=True)

change_climate_data_gdf = gpd.GeoDataFrame(pd.merge(climate_data_1980, midwest_counties_gdf, on='id', how='left'))

merged = gpd.GeoDataFrame(pd.merge(yield_change, change_climate_data_gdf, on='id', how='inner'))
merged.set_geometry('geometry', inplace=True)

corn_merged = merged[merged['commodity_desc']=='CORN']
heatmap_df = corn_merged[['ann_avg_temp', 'ann_avg_precip', 'yield_past']]
heatmap = (
    alt.Chart(heatmap_df)
    .mark_rect()
    .encode(
        x=alt.X('ann_avg_temp:Q', bin=alt.Bin(maxbins=12), title='Temperature'),
        y=alt.Y('ann_avg_precip:Q', bin=alt.Bin(maxbins=12), title='Precipitation'),
        color=alt.Color('mean(yield_past):Q', scale=alt.Scale(scheme='viridis'), title='Present Yield (2018-2023)')
    )
    .properties(
        width=600,
        height=400,
        title=alt.TitleParams(
            text='Heat Map of Yield (1980)',
            subtitle='Crop Yield (BU/acre) by Climate  ',
            anchor='middle'
    ),
    )    
)



heatmap.display()

In [ ]:
climate_data_2023 = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([2023])]
climate_data_2023.rename(columns={'County_Code': 'id'}, inplace=True)

change_climate_data_gdf = gpd.GeoDataFrame(pd.merge(climate_data_2023, midwest_counties_gdf, on='id', how='left'))

merged = gpd.GeoDataFrame(pd.merge(yield_change, change_climate_data_gdf, on='id', how='inner'))
merged.set_geometry('geometry', inplace=True)

corn_merged = merged[merged['commodity_desc']=='CORN']
heatmap_df = corn_merged[['ann_avg_temp', 'ann_avg_precip', 'yield_present']]
heatmap = (
    alt.Chart(heatmap_df)
    .mark_rect()
    .encode(
        x=alt.X('ann_avg_temp:Q', bin=alt.Bin(maxbins=12), title='Temperature'),
        y=alt.Y('ann_avg_precip:Q', bin=alt.Bin(maxbins=12), title='Precipitation'),
        color=alt.Color('mean(yield_present):Q', scale=alt.Scale(scheme='viridis'), title='Present Yield (2018-2023)')
    )
    .properties(
        width=600,
        height=400,
        title=alt.TitleParams(
            text='Heat Map of Yield Change (1975-2023)',
            subtitle='Average change in Crop Yield (BU/acre) for Counties Grouped by Long/Lat ',
            anchor='middle'
    ),
    )
)


heatmap.display()

In [ ]:
change_climate_data_gdf = gpd.GeoDataFrame(pd.merge(climate_data_1980_2023, midwest_counties_gdf, on='id', how='left'))
change_climate_data_gdf

merged = gpd.GeoDataFrame(pd.merge(yield_change, change_climate_data_gdf, on='id', how='inner'))
merged.set_geometry('geometry', inplace=True)

merged = merged[['commodity_desc','abs_change_yield', 'ann_avg_temp_x', 'ann_avg_temp_y', 'geometry']]

corn_merged = merged[merged['commodity_desc']=='CORN']
heatmap_df = corn_merged[['ann_avg_temp_x', 'ann_avg_temp_y', 'abs_change_yield']]

# Filter
heatmap_df = heatmap_df[(heatmap_df['ann_avg_temp_x'] >= 44) & (heatmap_df['ann_avg_temp_x'] <= 56)]
heatmap_df = heatmap_df[(heatmap_df['ann_avg_temp_y'] >= 44) & (heatmap_df['ann_avg_temp_y'] <= 56)]


heatmap = (
    alt.Chart(heatmap_df)
    .mark_rect()
    .encode(
        x=alt.X('ann_avg_temp_x:Q', bin=alt.Bin(maxbins=30), title='Temperature 1980'),
        y=alt.Y('ann_avg_temp_y:Q', bin=alt.Bin(maxbins=30), title='Temp 2023'),
        color=alt.Color('mean(abs_change_yield):Q', scale=alt.Scale(scheme='viridis'), title='Present Yield (2018-2023)')
    )
    .properties(
        width=600,
        height=400,
        title=alt.TitleParams(
            text='Heat Map of Yield Change (1975-2023)',
            subtitle='Average change in Crop Yield (BU/acre) for Counties Grouped by Long/Lat ',
            anchor='middle'
    ),
    )
)


heatmap.display()

## Yield Map

In [ ]:
state_ansi_list = check.iloc[:,0].to_list()
midwest_counties_gdf = counties_gdf[counties_gdf['id'].str[:2].isin(state_ansi_list)]
midwest_counties_gdf = midwest_counties_gdf[
    counties_gdf['id'].str[:2].isin(state_ansi_list) & 
    (counties_gdf['id'].str.len() == 5)  
]

In [ ]:
db_name = 'field_crops.db'
table_prod = 'midwest_key_field_crops_cleaned'
table_area = 'midwest_area_planted_cleaned'

query = f"""
Select 
    avg(value) AS avg_prod_present,
    commodity_desc,
    state_alpha, 
    state_ansi|| county_ansi as id
from {table_prod} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 2018 and 2023
group by state_ansi|| county_ansi, commodity_desc
"""

conn = sqlite3.connect(db_name) 
avg_prod_present = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS avg_prod_past,
    commodity_desc,
    state_alpha,
    state_ansi|| county_ansi as id
from {table_prod} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 1975 and 1980
group by state_ansi|| county_ansi, commodity_desc
"""
conn = sqlite3.connect(db_name) 
avg_prod_past = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS avg_area_present,
    commodity_desc,
    state_alpha,
    state_ansi|| county_ansi as id
from {table_area} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 2018 and 2023
group by state_ansi|| county_ansi , commodity_desc
"""
conn = sqlite3.connect(db_name) 
avg_area_present = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS avg_area_past,
    commodity_desc,
    state_alpha,
    state_ansi|| county_ansi as id
from {table_area} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 1975 and 1980
group by state_ansi|| county_ansi , commodity_desc
"""
avg_area_past = pd.read_sql(query, conn)

avg_yield_past = pd.merge(avg_prod_past, avg_area_past, on=["commodity_desc", "id", "state_alpha"])
avg_yield_past["yield_past"] = (avg_yield_past['avg_prod_past'] / avg_yield_past['avg_area_past']) 

avg_yield_present = pd.merge(avg_prod_present, avg_area_present, on=["commodity_desc", "id", "state_alpha"])
avg_yield_present["yield_present"] = (avg_yield_present['avg_prod_present'] / avg_yield_present['avg_area_present']) 

yield_change = pd.merge(avg_yield_past, avg_yield_present, on=["commodity_desc", "id", "state_alpha"])

yield_change["abs_change_yield"] = (yield_change['yield_present'] -  yield_change['yield_past']) 
yield_change["perc_change_yield"] = ((yield_change['yield_present'] -  yield_change['yield_past']) / yield_change['yield_past'])*100
yield_change

In [ ]:

query = f"""
Select 
    distinct
    state_ansi
from {crop_table} 
"""
conn = sqlite3.connect(db_name) 
check = pd.read_sql(query, conn)

state_ansi_list = check.iloc[:,0].to_list()
midwest_counties_gdf = counties_gdf[counties_gdf['id'].str[:2].isin(state_ansi_list)]
midwest_counties_gdf = midwest_counties_gdf[
    counties_gdf['id'].str[:2].isin(state_ansi_list) & 
    (counties_gdf['id'].str.len() == 5)  
]


merged = gpd.GeoDataFrame(pd.merge(yield_change, midwest_counties_gdf, on='id', how='left'))
merged.set_geometry('geometry', inplace=True)


crop_list = [ 'CORN', 'SOYBEANS', 'WHEAT']

for crop in crop_list:
    crop_df = merged[merged['commodity_desc']== crop]

    # Define the background chart with a gray fill and black stroke for county borders
    county_map_background = alt.Chart(midwest_counties_gdf).mark_geoshape(
        fill='lightgray',  # Background color
        stroke='black',    # Outline color for counties
        strokeWidth=0.5    # Thickness of county borders
    ).properties(
        width=800,
        height=500
    ).project('albersUsa')  # Use Albers USA projection

    # Define the filled map chart
    county_map_filled = alt.Chart(crop_df).mark_geoshape(
        stroke='black',   # Outline color for counties
        strokeWidth=0.5   # Thickness of county borders
    ).encode(
        color=alt.Color('yield_present:Q', scale=alt.Scale(scheme='redblue')),  # Sequential color scale for the 'value' column
        tooltip=['id:N', 'yield_present:Q']  # Tooltip with county ID and value
    ).properties(
        title=f'Map of Avg Yield for {crop}: Present (2018-2023)',
        width=800,
        height=500
    ).project('albersUsa')  # Use Albers USA projection

    # Layer the filled map on top of the gray background
    layered_map = county_map_background + county_map_filled

    # Display the chart
    layered_map.show()


In [ ]:

query = f"""
Select 
    distinct
    state_ansi
from {table} 
"""
conn = sqlite3.connect(db_name) 
check = pd.read_sql(query, conn)

state_ansi_list = check.iloc[:,0].to_list()
midwest_counties_gdf = counties_gdf[counties_gdf['id'].str[:2].isin(state_ansi_list)]
midwest_counties_gdf = midwest_counties_gdf[
    counties_gdf['id'].str[:2].isin(state_ansi_list) & 
    (counties_gdf['id'].str.len() == 5)  
]


merged = gpd.GeoDataFrame(pd.merge(yield_change, midwest_counties_gdf, on='id', how='left'))
merged.set_geometry('geometry', inplace=True)


crop_list = [ 'CORN', 'SOYBEANS', 'WHEAT']

for crop in crop_list:
    crop_df = merged[merged['commodity_desc']== crop]

    # Define the background chart with a gray fill and black stroke for county borders
    county_map_background = alt.Chart(midwest_counties_gdf).mark_geoshape(
        fill='lightgray',  # Background color
        stroke='black',    # Outline color for counties
        strokeWidth=0.5    # Thickness of county borders
    ).properties(
        width=800,
        height=500
    ).project('albersUsa')  # Use Albers USA projection

    # Define the filled map chart
    county_map_filled = alt.Chart(crop_df).mark_geoshape(
        stroke='black',   # Outline color for counties
        strokeWidth=0.5   # Thickness of county borders
    ).encode(
        color=alt.Color('abs_change_yield:Q',       
                        scale=alt.Scale(
                        scheme='redblue',        # Sequential color scale for yield change       # Adjust domain to match data range for more contrast
        )),  # Sequential color scale for the 'value' column
        tooltip=['id:N', 'abs_change_yield:Q']  # Tooltip with county ID and value
    ).properties(
        title=f'Map of Production for {crop}',
        width=800,
        height=500
    ).project('albersUsa')  # Use Albers USA projection

    # Layer the filled map on top of the gray background
    layered_map = county_map_background + county_map_filled

    # Display the chart
    layered_map.show()


In [ ]:
yield_change
corn_yield_change = yield_change[yield_change['commodity_desc'] == 'CORN']
corn_yield_change.describe()

In [ ]:
db_name = 'field_crops.db'
table = 'midwest_key_field_crops_cleaned'
version = 'pct'

#counties = data.us_10m.url  # URL for U.S. counties

midwest_counties_gdf = load_midwest_counties(db_name, table, counties_gdf)


pct_change_climate_data_df.rename(columns={'County_Code': 'id'}, inplace=True)

# Merge the result DataFrame with the GeoDataFrame
change_climate_data_gdf = gpd.GeoDataFrame(pd.merge(pct_change_climate_data_df, midwest_counties_gdf, on='id', how='left'))

# Set the geometry for the GeoDataFrame
change_climate_data_gdf.set_geometry('geometry', inplace=True)
print(len(change_climate_data_gdf))

climate_metrics = [f'ann_avg_precip_{version}_change', f'ann_avg_temp_{version}_change', f'ann_max_temp_{version}_change', f'ann_min_temp_{version}_change']
create_climate_maps(change_climate_data_gdf, midwest_counties_gdf, climate_metrics)

In [ ]:
abs_change_climate_data_df.rename(columns={'County_Code': 'id'}, inplace=True)
pct_change_climate_data_df.rename(columns={'County_Code': 'id'}, inplace=True)
#pct_change_climate_data_df.reset_index(drop=True, inplace=True)
# merge with county yield data
merged_pct_change = pd.merge(yield_change, pct_change_climate_data_df, on='id', how="left")
merged_abs_change = pd.merge(yield_change, abs_change_climate_data_df, on='id', how="left")
merged_abs_change

### 2023 climate data values

In [ ]:
climate_averages_2023 = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([2023])]
climate_averages_2023.rename(columns={'County_Code': 'id'}, inplace=True)
merged_2023 = pd.merge(yield_change, climate_averages_2023, on='id', how="left")
merged_2023

In [ ]:
climate_features = [ 'ann_avg_precip', 'ann_avg_temp', 'ann_max_temp', 'ann_min_temp']
# Create a scatter plot with facets for each commodity
corn_merged_2023  = merged_2023[merged_2023['commodity_desc']=='CORN']
for feature in climate_features:
    scatter = alt.Chart(corn_merged_2023).mark_circle(size=60).encode(
        x=alt.X(
            f'{feature}:Q',
                    scale=alt.Scale(domain=[merged_2023[f'{feature}'].min(), merged_2023[f'{feature}'].max()])  # Set x-axis to start at 30
        ),
        y='yield_present:Q'
    ).properties(
        title='Scatter Plot with Quadratic Best Fit Line'
    )

    line_of_best_fit = scatter.transform_regression(
    f'{feature}', 'yield_present', method="poly",
    ).transform_calculate(
        ann_avg_temp_squared=f'datum.{feature} * datum.{feature}'
    ).mark_line(color='red')


    # Combine the scatter plot and line of best fit, and facet by commodity_desc
    chart = scatter + line_of_best_fit



    # Display the chart
    chart.show()



In [ ]:
chart = alt.Chart(corn_merged_2023).mark_bar().encode(
    x=alt.X('ann_avg_temp', bin=alt.Bin(maxbins=30)),
    y=alt.Y('count():Q', title='Average Value'),
    tooltip=['ann_avg_temp', 'mean(yield_present)']
)

hist = alt.Chart(corn_merged_2023).mark_bar().encode(
    x=alt.X('ann_avg_temp', bin=alt.Bin(maxbins=30)),
    y='count()'
)

hist

In [ ]:
chart = alt.Chart(corn_merged_2023).mark_bar().encode(
    x=alt.X('ann_avg_temp', bin=alt.Bin(maxbins=30)),
    y=alt.Y('mean(yield_present):Q', title='Average Value'),
    tooltip=['ann_avg_temp', 'mean(yield_present)']
)
chart

### 1980 Yiel Scatter

In [ ]:
climate_data_1980 = rolling_avg_30yr_climate_data_df[rolling_avg_30yr_climate_data_df['Year'].isin([1980])]
climate_data_1980.rename(columns={'County_Code': 'id'}, inplace=True)

change_climate_data_gdf = gpd.GeoDataFrame(pd.merge(climate_data_1980, midwest_counties_gdf, on='id', how='left'))

merged = gpd.GeoDataFrame(pd.merge(yield_change, change_climate_data_gdf, on='id', how='inner'))
merged.set_geometry('geometry', inplace=True)


# Create a scatter plot with facets for each commodity
scatter = alt.Chart(merged).mark_circle(size=60).encode(
    x=alt.X(
        'ann_avg_temp:Q',  # Quantitative x-axis
        scale=alt.Scale(domain=[30, merged['ann_avg_temp'].max()])  # Set x-axis to start at 30
    ),
    y='yield_past:Q'
).properties(
    title='Scatter Plot with Quadratic Best Fit Line'
)

# Create a quadratic line of best fit by using linear regression on ann_avg_temp and ann_avg_temp_squared
line_of_best_fit = scatter.transform_regression(
    'ann_avg_temp', 'yield_past', method="poly",
).transform_calculate(
    ann_avg_temp_squared='datum.ann_avg_temp * datum.ann_avg_temp'
).mark_line(color='red')

# Combine the scatter plot and line of best fit, and facet by commodity_desc
chart = scatter + line_of_best_fit

# Facet the chart by 'commodity_desc'
faceted_chart = chart.facet(
    facet='commodity_desc:N',  # Use nominal encoding for faceting
    columns=3  # Number of columns for the facets
)

# Display the chart
faceted_chart.show()


In [ ]:
# Create a scatter plot with facets for each commodity
scatter = alt.Chart(merged).mark_circle(size=60).encode(
    x=alt.X(
        'ann_avg_precip:Q',  # Quantitative x-axis
        #scale=alt.Scale(domain=[30, merged['ann_avg_precip'].max()])  # Set x-axis to start at 30
    ),
    y='yield_past:Q'
).properties(
    title='Scatter Plot with Quadratic Best Fit Line'
)

# Create a quadratic line of best fit by using linear regression on ann_avg_temp and ann_avg_temp_squared
line_of_best_fit = scatter.transform_regression(
    'ann_avg_precip', 'yield_past', method="poly",
).transform_calculate(
    ann_avg_temp_squared='datum.ann_avg_precip * datum.ann_avg_precip'
).mark_line(color='red')

# Combine the scatter plot and line of best fit, and facet by commodity_desc
chart = scatter + line_of_best_fit

# Facet the chart by 'commodity_desc'
faceted_chart = chart.facet(
    facet='commodity_desc:N',  # Use nominal encoding for faceting
    columns=3  # Number of columns for the facets
)

# Display the chart
faceted_chart.show()

### pct change in climate data

In [ ]:
# Create a scatter plot with facets for each commodity
scatter = alt.Chart(merged_pct_change).mark_circle(size=60).encode(
    x='ann_avg_temp_pct_change:Q',  # Quantitative x-axis
    y='abs_change_yield:Q',  # Quantitative y-axis
    tooltip=['ann_avg_temp_pct_change', 'abs_change_yield']  # Tooltip showing values on hover
).properties(
    title='Scatter Plot with Line of Best Fit'
)

# Create a line of best fit (linear regression) and facet by commodity_desc
line_of_best_fit = scatter.transform_regression('ann_avg_temp_pct_change', 'abs_change_yield').mark_line(color='red')

# Combine the scatter plot and line of best fit, and facet by commodity_desc
chart = scatter + line_of_best_fit

# Facet the chart by 'commodity_desc'
faceted_chart = chart.facet(
    facet='commodity_desc:N',  # Use nominal encoding for faceting
    columns=3  # Number of columns for the facets
)

# Display the chart
faceted_chart.show()

In [ ]:
# Create a scatter plot with facets for each commodity
scatter = alt.Chart(merged_pct_change).mark_circle(size=60).encode(
    x='ann_avg_precip_pct_change:Q',  # Quantitative x-axis
    y='abs_change_yield:Q',  # Quantitative y-axis
    tooltip=['ann_avg_precip_pct_change', 'abs_change_yield']  # Tooltip showing values on hover
).properties(
    title='Scatter Plot with Line of Best Fit'
)

# Create a line of best fit (linear regression) and facet by commodity_desc
line_of_best_fit = scatter.transform_regression('ann_avg_precip_pct_change', 'abs_change_yield').mark_line(color='red')

# Combine the scatter plot and line of best fit, and facet by commodity_desc
chart = scatter + line_of_best_fit

# Facet the chart by 'commodity_desc'
faceted_chart = chart.facet(
    facet='commodity_desc:N',  # Use nominal encoding for faceting
    columns=3  # Number of columns for the facets
)

# Display the chart
faceted_chart.show()

In [ ]:
# Create a scatter plot with facets for each commodity
scatter = alt.Chart(merged_pct_change).mark_circle(size=60).encode(
    x='ann_max_temp_pct_change:Q',  # Quantitative x-axis
    y='abs_change_yield:Q',  # Quantitative y-axis
    tooltip=['ann_max_temp_pct_change', 'abs_change_yield']  # Tooltip showing values on hover
).properties(
    title='Scatter Plot with Line of Best Fit'
)

# Create a line of best fit (linear regression) and facet by commodity_desc
line_of_best_fit = scatter.transform_regression('ann_max_temp_pct_change', 'abs_change_yield').mark_line(color='red')

# Combine the scatter plot and line of best fit, and facet by commodity_desc
chart = scatter + line_of_best_fit

# Facet the chart by 'commodity_desc'
faceted_chart = chart.facet(
    facet='commodity_desc:N',  # Use nominal encoding for faceting
    columns=3  # Number of columns for the facets
)

# Display the chart
faceted_chart.show()


In [ ]:
# Create a scatter plot with facets for each commodity
scatter = alt.Chart(merged_pct_change).mark_circle(size=60).encode(
    x='ann_min_temp_pct_change:Q',  # Quantitative x-axis
    y='abs_change_yield:Q',  # Quantitative y-axis
    tooltip=['ann_min_temp_pct_change', 'abs_change_yield']  # Tooltip showing values on hover
).properties(
    title='Scatter Plot with Line of Best Fit'
)

# Create a line of best fit (linear regression) and facet by commodity_desc
line_of_best_fit = scatter.transform_regression('ann_min_temp_pct_change', 'abs_change_yield').mark_line(color='red')

# Combine the scatter plot and line of best fit, and facet by commodity_desc
chart = scatter + line_of_best_fit

# Facet the chart by 'commodity_desc'
faceted_chart = chart.facet(
    facet='commodity_desc:N',  # Use nominal encoding for faceting
    columns=3  # Number of columns for the facets
)

# Display the chart
faceted_chart.show()


### abs change in climate data

In [ ]:
# Create a scatter plot with facets for each commodity
scatter = alt.Chart(merged_abs_change).mark_circle(size=60).encode(
    x='ann_avg_temp_abs_change:Q',  # Quantitative x-axis
    y='abs_change_yield:Q',  # Quantitative y-axis
    tooltip=['ann_avg_temp_abs_change', 'abs_change_yield']  # Tooltip showing values on hover
).properties(
    title='Scatter Plot with Line of Best Fit'
)

# Create a line of best fit (linear regression) and facet by commodity_desc
line_of_best_fit = scatter.transform_regression('ann_avg_temp_abs_change', 'abs_change_yield').mark_line(color='red')

# Combine the scatter plot and line of best fit, and facet by commodity_desc
chart = scatter + line_of_best_fit

# Facet the chart by 'commodity_desc'
faceted_chart = chart.facet(
    facet='commodity_desc:N',  # Use nominal encoding for faceting
    columns=3  # Number of columns for the facets
)

# Display the chart
faceted_chart.show()

## heat map by climate features

In [ ]:


merged_abs_change[['commodity_desc','abs_change_yield', 'ann_avg_temp_abs_change', 'ann_avg_precip_abs_change']]

merged_abs_change

In [ ]:



heatmap_df = merged_abs_change[['commodity_desc','abs_change_yield', 'ann_avg_temp_abs_change', 'ann_avg_precip_abs_change']]

heatmap = (
    alt.Chart(heatmap_df)
    .mark_rect()
    .encode(
        x=alt.X('ann_avg_temp_abs_change:Q', bin=alt.Bin(maxbins=15), title='Absolute Change in Average Temperature'),
        y=alt.Y('ann_avg_precip_abs_change:Q', bin=alt.Bin(maxbins=15), title='Absolute Change in Average Precipitation'),
        color=alt.Color('mean(abs_change_yield):Q', scale=alt.Scale(scheme='viridis'), title='Absolute Change in Yield'),
        tooltip=['ann_avg_temp_abs_change:Q', 'latitude:Q', 'mean(abs_change_yield):Q']
    )
    .properties(
        width=600,
        height=400,
        title=alt.TitleParams(
            text='Heat Map of Yield Change (1975-2023)',
            subtitle='Average change in Crop Yield (BU/acre) for Counties Grouped by change in climate features ',
            anchor='middle'
    ),
    )
    .facet(
        facet=alt.Facet('commodity_desc:N', title='Commodity Type'),
        columns=3  # Adjust the number of columns as needed
    )
    .resolve_scale(
        color='independent'  # Allows each facet to have its own color scale
    )
)



heatmap.display()

In [ ]:
merged_2023

In [ ]:

heatmap_df = merged_2023

heatmap = (
    alt.Chart(heatmap_df)
    .mark_rect()
    .encode(
        x=alt.X('ann_avg_temp:Q', bin=alt.Bin(maxbins=15), title='Absolute Change in Average Temperature'),
        y=alt.Y('ann_avg_precip:Q', bin=alt.Bin(maxbins=15), title='Absolute Change in Average Precipitation'),
        color=alt.Color('mean(abs_change_yield):Q', scale=alt.Scale(scheme='viridis'), title='Absolute Change in Yield'),
        tooltip=['ann_avg_temp:Q', 'ann_avg_precip:Q', 'mean(abs_change_yield):Q']
    )
    .properties(
        width=600,
        height=400,
        title=alt.TitleParams(
            text='Heat Map of Yield Change (1975-2023)',
            subtitle='Average change in Crop Yield (BU/acre) for Counties Grouped by change in climate features ',
            anchor='middle'
    ),
    )
    .facet(
        facet=alt.Facet('commodity_desc:N', title='Commodity Type'),
        columns=3  # Adjust the number of columns as needed
    )
    .resolve_scale(
        color='independent'  # Allows each facet to have its own color scale
    )
)



heatmap.display()